In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


In [4]:
df = pd.read_csv('retractions_error_of_analyses.csv')
df.tail(15)

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes
1344,285,The solution structure of heparan sulfate diff...,(BLS) Biology - Molecular;(PHY) Chemistry;(PHY...,Department of Structural and Molecular Biology...,The Journal of Biological Chemistry,American Society for Biochemistry and Molecula...,United Kingdom,Sanaullah Khan;Elizabeth Rodriguez;Rima Patel;...,http://retractionwatch.com/2013/07/29/retracti...,Research Article;,7/19/2013 0:00,10.1074/jbc.A111.226027,23873987.0,5/16/2011 0:00,10.1074/jbc.M111.226027,21576246.0,Retraction,+Error in Analyses;+Error in Data;,No,NaN
1345,272,Over-expression of HSPA12B protects mice again...,(BLS) Biology - Cellular;(BLS) Biology - Molec...,Department of Thoracic and Cardiovascular Surg...,American Journal of Translational Research,e-Century Publishing Corporation,China,Yanjun Sun;Lincai Ye;Chuan Jiang;Jun Jiang;Hai...,NaN,Research Article;,3/15/2016 0:00,NaN,27186288.0,12/15/2015 0:00,NaN,26885270.0,Retraction,+Concerns/Issues About Data;+Error in Analyses...,No,NaN
1346,265,"Sedation with GPI 15715, a water-soluble prodr...",(HSC) Medicine - Anesthesia/Anaesthesia;(HSC) ...,"Department of Anesthesiology, University of Er...",Anesthesia & Analgesia,Wolters Kluwer - Lippincott Williams & Wilkins,Germany;United States,Jorg Fechner;Harald Ihmsen;Christine Schiessl;...,NaN,Clinical Study;,4/1/2010 0:00,10.1213/ANE.0b013e3181dbdb71,20357159.0,3/1/2005 0:00,10.1213/01.ANE.0000144772.13372.F4,15728055.0,Retraction,+Error by Third Party;+Error in Analyses;+Erro...,No,See also: https://journals.lww.com/anesthesia-...
1347,261,Cytosol has a small effect on protein backbone...,(BLS) Biochemistry;(BLS) Biology - Cellular;,"Department of Chemistry, Department of Biochem...",Biochemistry,American Chemical Society (ACS),United States,Julie E Bryant;Juliette T J Lecomte;Andrew L L...,NaN,Research Article;,6/13/2007 0:00,10.1021/bi700744h,17567051.0,8/1/2006 0:00,10.1021/bi060547b,1690676.0,Retraction,+Error in Analyses;+Error in Methods;+Results ...,No,NaN
1348,248,"N, S Co-doped Graphene Quantum Dots from a Sin...",(PHY) Chemistry;,Collaborative Innovation Center for Marine Bio...,Chemical Communications,Royal Society of Chemistry (RSC),China,Rijun Gui;Xifeng Liu;Hui Jin;Zonghua Wang;Feif...,http://retractionwatch.com/2015/05/27/scientis...,Research Article;,6/21/2015 0:00,10.1039/c5cc90244d,26008806.0,1/12/2015 0:00,10.1039/c4cc09280e,25578025.0,Retraction,+Concerns/Issues About Authorship/Affiliation;...,Yes,NaN
1349,247,Measuring Si-C60 Chemical Forces via Single Mo...,(PHY) Chemistry;,"School of Physics and Astronomy, University of...",Chemical Communications,Royal Society of Chemistry (RSC),United Kingdom,Cristina Chiutu;Andrew Stannard;Adam M Sweetma...,NaN,Research Article;,8/28/2014 0:00,10.1039/C4CC90295E,25055854.0,10/14/2011 0:00,10.1039/c1cc14147c,21869962.0,Retraction,+Error in Analyses;+Error in Results and/or Co...,Yes,NaN
1350,232,A Novel function of Poly(ADP-ribose) Polymeras...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,"Department of Community, Occupational & Family...",Cell Death and Differentiation,Nature Publishing Group,Singapore,Qing Huang;Y-T Wu;H-L Tan;C-N Ong;Han-Ming Shen,http://retractionwatch.com/2010/11/18/what-wer...,Research Article;,12/1/2010 0:00,10.1038/cdd.2010.121,20865014.0,2/16/2009 0:00,10.1038/cdd.2008.151,18974775.0,Retraction,+Error in Analyses;+Error in Data;,No,NaN
1351,231,Induced Endothelial Cells Enhance Osteogenesis...,(BLS) Biology - Cellular;(BLS) Biology - Gener...,"Department of Orthopedics, Shanghai First Peop...","Cells, Tissues, Organs in Vivo, in Vitro (CTO)",Karger,China,Jie Tao;Yuan Sun;Qui-gen Wang,NaN,Research Article;,4/13/2010 0:00,10.1159/000313340,20408260.0,10/18/2008 0:00,10.1159/000218139,19420896.0,Retraction,+Error in Analyses;+Error in Data;+Re

In [5]:
print(df.shape)

(1359, 20)


In [6]:
df_paywalled_no = df[df['Paywalled'] == 'No']
df_paywalled_yes = df[df['Paywalled'] == 'Yes']

In [7]:
print(df_paywalled_yes.shape)
print(df_paywalled_no.shape)

(49, 20)
(1304, 20)


In [14]:
def get_pdf_from_doi(doi):
    doi_url = f"https://doi.org/{doi}"
    response = requests.get(doi_url)

    if response.status_code != 200:
        print(f"Failed to resolve DOI: {doi}")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    pdf_link = None

    for link in soup.find_all('a'):
        href = link.get('href', '')
        if 'pdf' in href.lower():
            pdf_link = href
            break
    
    if not pdf_link:
        print(f"Failed to find PDF link for DOI: {doi}")
        return None
    
    if not pdf_link.startswith('http'):
        pdf_link = response.url.rsplit('/', 1)[0] + '/' + pdf_link

    print(f"Downloading PDF from {pdf_link}")
    pdf_response = requests.get(pdf_link)

    if pdf_response.status_code == 200:
        pdf_filename = f"{doi.replace('/', '_')}.pdf"
        with open(pdf_filename, 'wb') as f:
            f.write(pdf_response.content)
        print(f"PDF downloaded successfully: {pdf_filename}")
    else:
        print(f"Failed to download PDF: {pdf_link}")



In [8]:
def download_pdfs_from_df(df, column_name):
    for index, row in df.iterrows():
        doi = row[column_name]
        if pd.notna(doi):  # Check if DOI is not NaN
            print(f"Processing DOI: {doi}")
            get_pdf_from_doi(doi)
        else:
            print(f"No DOI found for index {index}")

In [16]:
download_pdfs_from_df(df_paywalled_no, 'RetractionDOI')

Processing DOI: 10.1016/j.clnu.2022.04.021
Failed to find PDF link for DOI: 10.1016/j.clnu.2022.04.021
Processing DOI: 10.1177/00220426221121697
Failed to resolve DOI: 10.1177/00220426221121697
Processing DOI: 10.1177/00220426231222323
Failed to resolve DOI: 10.1177/00220426231222323
Processing DOI: 10.1093/jnci/djae217
Failed to resolve DOI: 10.1093/jnci/djae217
Processing DOI: 10.1186/s12889-024-19943-3
Failed to download PDF: https://bmcpublichealth.biomedcentral.com/articles/10.1186///bmcpublichealth.biomedcentral.com/counter/pdf/10.1186/s12889-024-19943-3.pdf
Processing DOI: 10.1371/journal.pone.0309771
Failed to find PDF link for DOI: 10.1371/journal.pone.0309771
Processing DOI: 10.1093/jeb/voae081
Failed to resolve DOI: 10.1093/jeb/voae081
Processing DOI: 10.1001/jamapediatrics.2024.2996
Failed to resolve DOI: 10.1001/jamapediatrics.2024.2996
Processing DOI: 10.1126/science.abb4382
Failed to resolve DOI: 10.1126/science.abb4382
Processing DOI: 10.1126/science.adr6434
Failed to r

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [1]:
import requests

def get_paper_by_doi(doi):
    # Define the base URL for Europe PMC's RESTful Web Service
    base_url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
    
    # Set up parameters for the query
    params = {
        "query": f"DOI:{doi}",  # Search by DOI
        "format": "json",        # Specify JSON format for the response
        "resultType": "core"     # Get full metadata for the publication
    }
    
    # Send a GET request to the API
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Check if there are results
        if data.get("resultList", {}).get("result"):
            return data["resultList"]["result"][0]  # Return the first result
        else:
            return f"No results found for DOI: {doi}"
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
doi = "10.1074/jbc.A111.226027"  # Replace with the DOI you are interested in
paper_info = get_paper_by_doi(doi)
print(paper_info)


{'id': '23873987', 'source': 'MED', 'pmid': '23873987', 'pmcid': 'PMC3774387', 'fullTextIdList': {'fullTextId': ['PMC3774387']}, 'doi': '10.1074/jbc.a111.226027', 'title': 'The solution structure of heparan sulfate differs from that of heparin: implications for function.', 'authorString': 'Khan S, Rodriguez E, Patel R, Gor J, Mulloy B, Perkins SJ.', 'authorList': {'author': [{'fullName': 'Khan S', 'firstName': 'Sanaullah', 'lastName': 'Khan', 'initials': 'S', 'authorId': {'type': 'ORCID', 'value': '0000-0003-2480-6239'}}, {'fullName': 'Rodriguez E', 'firstName': 'Elizabeth', 'lastName': 'Rodriguez', 'initials': 'E'}, {'fullName': 'Patel R', 'firstName': 'Rima', 'lastName': 'Patel', 'initials': 'R'}, {'fullName': 'Gor J', 'firstName': 'Jayesh', 'lastName': 'Gor', 'initials': 'J'}, {'fullName': 'Mulloy B', 'firstName': 'Barbara', 'lastName': 'Mulloy', 'initials': 'B'}, {'fullName': 'Perkins SJ', 'firstName': 'Stephen J', 'lastName': 'Perkins', 'initials': 'SJ'}]}, 'authorIdList': {'autho

In [9]:
import requests
import pandas as pd

# Assuming 'df' is your DataFrame with a 'RetractionDOI' column
def get_paper_by_doi(doi):
    base_url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
    params = {
        "query": f"DOI:{doi}",
        "format": "json",
        "resultType": "core"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("resultList", {}).get("result"):
            return data["resultList"]["result"][0]  # Return first result
        else:
            return None
    else:
        print(f"Error {response.status_code} for DOI {doi}")
        return None

# Fetch metadata for each DOI in the DataFrame and store results
def get_retraction_metadata(df_paywalled_no):
    metadata_results = []
    for doi in df_paywalled_no["RetractionDOI"].dropna():  # Skip NaN values
        metadata = get_paper_by_doi(doi)
        if metadata:
            metadata_results.append(metadata)
    
    # Convert results to DataFrame if needed
    metadata_df = pd.DataFrame(metadata_results)
    return metadata_df

# Usage
metadata_df = get_retraction_metadata(df)
print(metadata_df)


           id source      pmid                           doi  \
0    39343464    MED  39343464    10.1016/j.clnu.2022.04.021   
1    39267576    MED  39267576          10.1093/jnci/djae217   
2    39237928    MED  39237928    10.1186/s12889-024-19943-3   
3    39197035    MED  39197035  10.1371/journal.pone.0309771   
4    39145387    MED  39145387           10.1093/jeb/voae081   
..        ...    ...       ...                           ...   
914  17990378    MED  17990378    10.1016/j.cell.2007.01.001   
915  16220614    MED  16220614    10.1016/j.cell.2005.08.028   
916  22719271    MED  22719271     10.1107/s1600536812022787   
917  17567051    MED  17567051             10.1021/bi700744h   
918  26310234    MED  26310234        10.1021/acsami.5b07490   

                                                 title  \
0    Retraction notice to 'High protein diet improv...   
1    Retraction and Replacement of: Increasing powe...   
2    Retraction Note: Association of milk consumpti...   

In [10]:
print(metadata_df.columns)

Index(['id', 'source', 'pmid', 'doi', 'title', 'authorString', 'authorList',
       'journalInfo', 'pubYear', 'pageInfo', 'affiliation',
       'publicationStatus', 'language', 'pubModel', 'pubTypeList',
       'subsetList', 'fullTextUrlList', 'commentCorrectionList',
       'isOpenAccess', 'inEPMC', 'inPMC', 'hasPDF', 'hasBook', 'hasSuppl',
       'citedByCount', 'hasData', 'hasReferences', 'hasTextMinedTerms',
       'hasDbCrossReferences', 'hasLabsLinks', 'hasEvaluations', 'authMan',
       'epmcAuthMan', 'nihAuthMan', 'hasTMAccessionNumbers', 'dateOfCreation',
       'firstIndexDate', 'dateOfRevision', 'firstPublicationDate', 'pmcid',
       'fullTextIdList', 'dataLinksTagsList', 'license',
       'fullTextReceivedDate', 'electronicPublicationDate', 'authorIdList',
       'abstractText', 'dateOfCompletion', 'meshHeadingList', 'chemicalList',
       'grantsList', 'manuscriptId', 'bookOrReportDetails',
       'tmAccessionTypeList', 'keywordList', 'dbCrossReferenceList',
       'embar

In [11]:
nan_titles = df[df['Title'].isna()]
print(nan_titles)

Empty DataFrame
Columns: [Record ID, Title, Subject, Institution, Journal, Publisher, Country, Author, URLS, ArticleType, RetractionDate, RetractionDOI, RetractionPubMedID, OriginalPaperDate, OriginalPaperDOI, OriginalPaperPubMedID, RetractionNature, Reason, Paywalled, Notes]
Index: []
